In [24]:
import numpy as np
import pandas as pd
import urllib

# Scrap neighborhood information from Wikipedia

The first step is to request the URL with the information and reading all the content into a variable

In [15]:
page = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
content = page.read()

The content is in bytes format so there is need to decode into a string in UTF-8 format

In [ ]:
scontent = content.decode("UTF-8")

As there is only one table in the webpage, it is very easy to extract by just looking for the the _<table_ and _</table>_ tags in the content, and selecting everything among them.

In [ ]:
tabla = scontent[scontent.find("<table"):scontent.find("</table>")+8]

Pandas is able to read a table from HTML so just call the reader

In [ ]:
datos = pd.read_html(tabla, header = 0)[0]

I have not used Beautiful Soup because it look like an overkill for this specific task

And now, select only those Boroughs that have an assigned name, and assign the Borough name to those neighbourhoods with no assigned name.

In [46]:
datos = datos[datos.Borough != "Not assigned"]
datos.Neighbourhood[datos.Neighbourhood == "Not assigned"] = datos.Borough[datos.Neighbourhood == "Not assigned"]
datos.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Now let us define a function (*concatenate_neighbourhood*) that takes a series of neighbourhood names and combines them properly with commas in between names, taking care that there is no comma after the last name. Let us also define a function that take the list of Borough associated with a postcode and verified that the postcode is completely contained within the Borough. If it is not, launch an expcetion. If it is, return the name of the Borough.

In [58]:
def concatenate_neighbourhood(x):
    cadena = ""
    for i in range(len(x)-1):
        cadena = cadena + x.iloc[i] + ", "
    cadena += x.iloc[-1]
    return cadena

def select_Borough(x):
    ref = x.iloc[0]
    for i in range(1, len(x)):
        if ref != x.iloc[i]:
            for i in x:
                print(x)
            raise Exception("Postcode comprises two Boroughs")
    return ref

Let us now group all the data by **Postcode**, to compose the name of all neighbourhood.

In [59]:
pdatos = datos.groupby(["Postcode"]).agg({"Borough": lambda x: select_Borough(x),
                                 "Neighbourhood": lambda x: concatenate_neighbourhood(x)})
pdatos.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


And the final size of my dataframe is:

In [51]:
pdatos.shape

(103, 2)